In [1]:
!nvidia-smi

Wed Aug  7 16:56:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8               1W /  30W |     90MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate



   ---------------------------------------- 15.8/15.8 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
Found existing installation: transformers 4.43.4
Uninstalling transformers-4.43.4:
  Successfully uninstalled transformers-4.43.4
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
   ---------------------------------------- 9.5/9.5 MB 3.4 MB/s eta 0:00:00
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)


In [5]:
from transformers import pipeline,set_seed
from datasets import load_dataset,load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_ckpt='google/pegasus-cnn_dailymail'

tokenizer=AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

c:\Users\Asus\Desktop\TextSummarization\TextSummarization\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Asus\Desktop\TextSummarization\TextSummarization\venv\lib\site-pa

In [16]:
dataset_samsum=load_dataset("Samsung/samsum",trust_remote_code=True)
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [17]:
split_lengths=[len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split Lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split Lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [18]:
def convert_examples_to_features(example_batch):
    input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)

    return{
        'input_ids':input_encodings['input_ids'],
        'attention_mask':input_encodings['attention_mask'],
        'labels':target_encodings['input_ids']
    }

In [19]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Asus\Desktop\TextSummarization\TextSummarization\venv\lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 5143.33 examples/s]


In [20]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [21]:
dataset_samsum_pt['train'][1]

{'id': '13728867',
 'dialogue': 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 'summary': 'Olivia and Olivier are voting for liberals in this election. ',
 'input_ids': [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]}

In [22]:
# TRAINING

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [39]:
from transformers import TrainingArguments,Trainer

trainer_args=TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,per_device_train_batch_size=1,
    per_device_eval_batch_size=1,weight_decay=0.01,logging_steps=10,evaluation_strategy='steps',eval_steps=500,
    save_steps=1e12,gradient_accumulation_steps=128
)

In [40]:
trainer=Trainer(
    model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt['test'],eval_dataset=dataset_samsum_pt['validation']
)

In [42]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 376.00 MiB (GPU 0; 4.00 GiB total capacity; 6.03 GiB already allocated; 0 bytes free; 6.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [41]:
import torch
torch.cuda.empty_cache()


In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements,batch_size):
    for i in range(0,len(list_of_elements),batch_size):
        yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,batch_size=16,device=device,
                                column_text="article",column_summary="highlights"):
    article_batches=list(generate_batch_sized_chunks(dataset[column_text],batch_size))
    target_batches=list(generate_batch_sized_chunks([column_summary],batch_size))

    for article_batch,target_batch in tqdm(zip(article_batches,target_batches),total=len(article_batches)):

        inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")

        summaries=model.generate(input_ids=inputs["input_ids"].to(device),attention_mask=inputs["attention_mask"].to(device),
                                 length_penalty=0.8,num_beams=8,max_length=128)
        

        decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]

        decoded_summaries=[d.replace(""," ") for d in decoded_summaries]

    
    score=metric.compute()
    return score



In [ ]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric=load_metric("rouge")

In [ ]:
score=calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size=2,
    column_text='dialogue',column_summary='summary'
)

rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn in rouge_names) 

pd.DataFrame(rouge_dict,index=[f'pegasus'])

In [ ]:
# Save Model

model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
# save tokenizer
tokenizer..save_pretrained("tokenizer")

In [ ]:
#Load

tokenizer=AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
# Prediction

gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":128}

sample_text=dataset_samsum['test'][0]["dialogue"]
reference=dataset_samsum['test'][0]["summary"]

pipe=pipeline("summarization",model="pegasus-samsum-model",tokenizer=tokenizer)

print("Dialogue")
print(sample_text)

print("\nReference Summary")
print(reference)

print("\nModel summary")
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])